In [40]:
%pip install langchain openai

Note: you may need to restart the kernel to use updated packages.


LLMs

In [41]:
import os

os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

Chains

In [42]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(model='gpt-3.5-turbo-0125',openai_api_key=os.environ['OPENAI_API_KEY'])
chat_model.predict("Who is prime minister of India?")

'Narendra Modi is the current Prime Minister of India.'

In [43]:
chat_model.predict("What was my previous question?")

'Your previous question was "What is your favorite color?"'

In [44]:
from langchain.chains import ConversationChain

chain = ConversationChain(llm=chat_model,verbose=True)

chain.run('who is founder of google?')




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: who is founder of google?
AI:

> Finished chain.


'The founders of Google are Larry Page and Sergey Brin. They started Google as a research project at Stanford University in 1996 and officially launched it in 1998.'

In [45]:
chain.run('What was my previous question?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: who is founder of google?
AI: The founders of Google are Larry Page and Sergey Brin. They started Google as a research project at Stanford University in 1996 and officially launched it in 1998.
Human: What was my previous question?
AI:



> Finished chain.


'Your previous question was "who is founder of google?"'

In [46]:
from langchain.prompts import PromptTemplate

template = """ Return all the sub-category of the category 

{category}
"""

prompt = PromptTemplate(input_variables=["category"], template=template)

prompt

PromptTemplate(input_variables=['category'], template=' Return all the sub-category of the category \n\n{category}\n')

In [47]:
from langchain.chains import LLMChain

chain = LLMChain(llm=chat_model, prompt=prompt, verbose=True)

chain.run(category="Machine Learning")



> Entering new LLMChain chain...
Prompt after formatting:
 Return all the sub-category of the category 

Machine Learning


> Finished chain.


'1. Supervised Learning\n2. Unsupervised Learning\n3. Reinforcement Learning\n4. Deep Learning\n5. Transfer Learning\n6. Online Learning\n7. Semi-supervised Learning\n8. Ensemble Learning\n9. Bayesian Learning\n10. Inductive Learning'

In [48]:
from langchain.prompts import( SystemMessagePromptTemplate,
                              HumanMessagePromptTemplate,
                              ChatPromptTemplate)

system_template = """You are an assitant who helps in finding sub-categories of a category
A user will only pass category and you have to generate comma separated list of sub-categories, nothing else.
"""

human_template = '{category}'

system_message = SystemMessagePromptTemplate.from_template(system_template)

system_message

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an assitant who helps in finding sub-categories of a category\nA user will only pass category and you have to generate comma separated list of sub-categories, nothing else.\n'))

In [49]:
human_message = HumanMessagePromptTemplate.from_template(human_template)
human_message

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['category'], template='{category}'))

In [50]:
prompt = ChatPromptTemplate.from_messages([system_message, human_message])

chain = LLMChain(llm=chat_model,
prompt=prompt,
verbose=True)

chain.run(category="Machine Learning")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an assitant who helps in finding sub-categories of a category
A user will only pass category and you have to generate comma separated list of sub-categories, nothing else.

Human: Machine Learning

> Finished chain.


'Supervised Learning, Unsupervised Learning, Reinforcement Learning, Deep Learning, Natural Language Processing, Computer Vision'

**OUTPUT Parser**


In [51]:
from langchain.schema import BaseOutputParser

class CommaSeparatedParser(BaseOutputParser):
    def parse(self, text):
        output = text.strip().split(',')
        output = [x.strip() for x in output]
        return output;

chain = LLMChain(llm=chat_model,prompt=prompt,output_parser=CommaSeparatedParser(),verbose=True)

chain.run(category="Machine Learning")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an assitant who helps in finding sub-categories of a category
A user will only pass category and you have to generate comma separated list of sub-categories, nothing else.

Human: Machine Learning

> Finished chain.


['Supervised Learning',
 'Unsupervised Learning',
 'Reinforcement Learning',
 'Deep Learning',
 'Natural Language Processing',
 'Computer Vision',
 'Clustering',
 'Classification',
 'Regression']

In [52]:
input_list=[{"category":"food"},{"category":"country"},{"category":"colors"}]

response = chain.apply(input_list)
response



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an assitant who helps in finding sub-categories of a category
A user will only pass category and you have to generate comma separated list of sub-categories, nothing else.

Human: food
Prompt after formatting:
System: You are an assitant who helps in finding sub-categories of a category
A user will only pass category and you have to generate comma separated list of sub-categories, nothing else.

Human: country
Prompt after formatting:
System: You are an assitant who helps in finding sub-categories of a category
A user will only pass category and you have to generate comma separated list of sub-categories, nothing else.

Human: colors

> Finished chain.


[{'text': ['fruits', 'vegetables', 'grains', 'dairy', 'protein', 'snacks']},
 {'text': ['Africa',
   'Asia',
   'Europe',
   'North America',
   'South America',
   'Australia']},
 {'text': ['red',
   'blue',
   'green',
   'yellow',
   'orange',
   'purple',
   'pink',
   'black',
   'white',
   'brown']}]

In [39]:
response[2]['text']

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'gray']

**SIMPLE SEQUENCE**

In [ ]:
title_prompt = """You are a writer. Given a subject, you job is to return a fun title for a play

Subject: {subject}
Title:"""